In [ ]:
from nn.layers import *
from nn.models import *
from nn.optimization import *

In [1]:
from nn.train import *
from ml.proj1_helpers import load_csv_data, create_csv_submission, predict_labels, build_poly

In [2]:
y_train, x_train, id_train = load_csv_data("data/train.csv")


In [3]:
y_test, x_test, id_test  = load_csv_data("data/test.csv")

In [4]:
y_pred, _, id_pred = load_csv_data("data/sample-submission.csv")

In [5]:
x_train_p = x_train
x_test_p = x_test

In [ ]:
import datetime

def pre_processing(x, tbd_value, switch_type = "mean"):
    # TODO optimize and try without 2 forloops
    start_time = datetime.datetime.now()
    D = len(x[0])
    res = x.copy()
    for i in range(D):
        x_col = x[:,i]
        x_i = [k if k != tbd_value else np.NaN for k in x_col]
        res[:,i] = x_i
        
        if switch_type == "mean":
            value = np.nanmean(x_i)
            mask = np.isnan(x_i)
            res[mask,i] = value

        if switch_type == "most":
            '''
            Returns the most common value
            '''
            u, indices = np.unique(x_i, return_inverse=True)
            value = u[np.argmax(np.bincount(indices))]
            assert value != np.nan, "most values are unknown in colum {}".format(i+1)
            mask = np.isnan(x_i)
            res[mask,i] = value
            
            
    end_time = datetime.datetime.now()
    exection_time = (end_time - start_time).total_seconds()
    print("Preprocessing : execution time={t:.3f} seconds".format(t=exection_time))
    
    return res

x_train_p = pre_processing(x_train, -999, switch_type = "mean")
x_test_p = pre_processing(x_test, -999, switch_type = "mean")

In [6]:


print(x_train.shape)
print(y_train.shape)

(250000, 30)
(250000,)


In [7]:
print(y_train[0:20])

[ 1. -1. -1. -1. -1. -1.  1.  1. -1.  1. -1. -1. -1. -1. -1.  1. -1.  1.
 -1. -1.]


In [8]:
print(x_test.shape)
print(y_test.shape)

(568238, 30)
(568238,)


In [9]:
print(y_pred.shape)

(568238,)


In [10]:
N_train = x_train.shape[0]
N_test  = x_test.shape[0]

In [11]:
# Neural Net

In [12]:
model = smallFullyConnected()
loss_func = cross_entropy()
optimizer = optimizer = SGD(model.net, 1e-2)
batch_size = 1000
max_epochs = 100

y_train[y_train == -1] = 0

losses, weights = train_net(y_train, x_train_p, model, loss_func, optimizer, batch_size, max_epochs, show_every = 50000)

(Iteration 1 / 25000.0) loss: 0.687630408442117
(Epoch 1 / 100) Training Accuracy: 0.657332
(Epoch 2 / 100) Training Accuracy: 0.68796
(Epoch 3 / 100) Training Accuracy: 0.700596
(Epoch 4 / 100) Training Accuracy: 0.703908
(Epoch 5 / 100) Training Accuracy: 0.708236
(Epoch 6 / 100) Training Accuracy: 0.709248
(Epoch 7 / 100) Training Accuracy: 0.710308
(Epoch 8 / 100) Training Accuracy: 0.712252
(Epoch 9 / 100) Training Accuracy: 0.711012
(Epoch 10 / 100) Training Accuracy: 0.714652
(Epoch 11 / 100) Training Accuracy: 0.676044
(Epoch 12 / 100) Training Accuracy: 0.720392
(Epoch 13 / 100) Training Accuracy: 0.712584
(Epoch 14 / 100) Training Accuracy: 0.730608
(Epoch 15 / 100) Training Accuracy: 0.714044
(Epoch 16 / 100) Training Accuracy: 0.733388
(Epoch 17 / 100) Training Accuracy: 0.729936
(Epoch 18 / 100) Training Accuracy: 0.700676
(Epoch 19 / 100) Training Accuracy: 0.747284
(Epoch 20 / 100) Training Accuracy: 0.753744
(Epoch 21 / 100) Training Accuracy: 0.728372
(Epoch 22 / 100) 

In [ ]:
y_pred = predict(model, x_test_p, batch_size)

y_pred[y_pred == 0] = -1

create_csv_submission(id_pred, y_pred, "data/submission_neural_net.csv")

In [ ]:
print(x_test.shape)
print(y_pred.shape)


In [ ]:
print(y_train.shape)
print(x_train.shape)

In [ ]:
labels = np.array([1, 0 ,1])
data = np.array([[1, 2, 3, 4], [5, 5, 5, 5] ,[1, 2, 3, 4]])

print("labels")
print(labels)
print("data")
print(data)

for a,b in batch_iter(labels, data, batch_size=1, num_batches=3, shuffle = False):
    print("label")
    print(a)
    print("data")
    print(b)
    print("\n")